In [1]:
# Native libraries
import os
import math
# Essential Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from fredapi import Fred
from sklearn.preprocessing import MinMaxScaler
# Algorithms
from statistics import mean, stdev, variance
from minisom import MiniSom
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth
from collections import Counter
from sklearn.decomposition import PCA
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


/home/dwight/miniconda3/lib/python3.8/site-packages/tslearn/clustering/kmeans.py:16: UserWarning: Scikit-learn <0.24 will be deprecated in a future release of tslearn
  warnings.warn(


In [2]:
csv = pd.read_csv("monthly_data.csv")


In [3]:
fred = Fred(api_key='3c6ba58d26525f17af95af4fabed24be')


In [4]:
csv2 = csv[50:-20]
csv2 = csv2.drop(['SMU06419406562200001.1', 'SMU21000006056170001SA.1'], axis=1)

In [38]:
data = csv2.dropna(axis="columns")[8::3].reset_index().drop(["index", "Unnamed: 0"], axis=1)
data

,UNRATE,CPIAUCSL,FEDFUNDS,CSUSHPINSA,M2SL,M1SL,PSAVERT,PAYEMS,INDPRO,TB3MS,...,ESTPIEAMP01GPM,BROSERPA158MFRBDAL,LARTTULA158MFRBDAL,LAUCN310910000000004,LAUCN271470000000004,LFWATTFECAM647N,LFWA55FECAM647S,LFWA55FEKRM647S,LFWA55MAAUM647N,LFWA55MAAUM647S
0,5.4,191.700,1.93,157.497,6399.4,1374.2,4.0,132503.0,97.6137,2.07,...,-0.202429,-1.713722,2.689583,12.0,712.0,12971900.0,1725500.0,2.143286e+06,1063251.0,1.062949e+06
1,5.4,192.400,2.50,161.924,6432.5,1371.1,3.2,133032.0,99.4639,2.54,...,-0.201410,1.561165,16.658304,25.0,983.0,13010400.0,1745200.0,2.138804e+06,1072885.0,1.073819e+06
2,5.1,193.600,3.00,169.544,6473.1,1366.0,3.5,133690.0,99.6033,2.84,...,-0.100604,1.090706,-4.866917,11.0,747.0,13057400.0,1763600.0,2.166473e+06,1081863.0,1.081815e+06
3,4.9,196.100,3.50,175.922,6569.8,1377.8,2.6,134498.0,99.9435,3.44,...,1.206030,9.230125,-3.218818,9.0,726.0,13110300.0,1782100.0,2.175409e+06,1090390.0,1.090723e+06
4,5.0,198.100,4.00,179.675,6654.5,1376.1,3.5,134993.0,100.3216,3.88,...,0.000000,2.638735,6.844361,8.0,713.0,13152700.0,1798800.0,2.175877e+06,1099186.0,1.098971e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,3.7,256.118,2.13,211.802,14938.8,3844.0,7.3,151108.0,109.8543,1.95,...,-0.223881,2.935424,-1.687233,13.0,642.0,15569800.0,2620100.0,3.998115e+06,1437108.0,1.438242e+06
60,3.6,257.989,1.55,212.222,15254.4,3955.6,7.5,151758.0,110.0388,1.54,...,0.150376,5.135172,-4.344529,11.0,505.0,15624700.0,2625300.0,4.015310e+06,1444387.0,1.443890e+06
61,3.5,258.824,1.58,213.295,15473.4,4027.6,8.3,152523.0,109.2966,1.52,...,-0.452489,4.145019,2.584733,13.0,752.0,15663700.0,2629400.0,4.020187e+06,1450121.0,1.449740e+06
62,13.3,255.942,0.05,218.569,17893.0,16275.9,24.7,132994.0,92.0613,0.13,...,-0.152555,58.708814,23.355997,10.0,1953.0,15693800.0,2633500.0,4.032882e+06,1453775.0,1.453747e+06


In [92]:
target = pd.read_csv("micron.csv").drop("date", axis=1)[::-1].reset_index().drop("index", axis=1)
target

,sales
0,1308
1,1054
2,1258
3,1362
4,1225
...,...
60,4797
61,5438
62,6056
63,5773


In [14]:
full = data.join(target)

In [15]:
names = full
scaler = MinMaxScaler()
full = MinMaxScaler().fit_transform(full)
som_x = som_y = math.ceil(math.sqrt(math.sqrt(len(full))))
full = np.transpose(full)

In [16]:
since = time.time()
som_x = som_y = math.ceil(math.sqrt(math.sqrt(len(full))))
# I didn't see its significance but to make the map square,
# I calculated square root of map size which is 
# the square root of the number of series
# for the row and column counts of som

som = MiniSom(som_x, som_y,len(full[0]), sigma=0.3, learning_rate = 0.1)

som.random_weights_init(full)
som.train(full, 50000)
time_elapsed = time.time() - since
print(f"SOM clustering executed in {time_elapsed // 60}m {time_elapsed % 60}s ")

SOM clustering executed in 0.0m 5.428215503692627s 


In [17]:
since = time.time()
pca = PCA(n_components=2)

mySeries_transformed = pca.fit_transform(full)

time_elapsed = time.time() - since
print(f"PCA clustering executed in {time_elapsed // 60}m {time_elapsed % 60}s ")

PCA clustering executed in 0.0m 0.3369779586791992s 


In [18]:
since = time.time()
cluster_count = math.ceil(math.sqrt(len(full))) 
print(cluster_count)
kmeans = KMeans(n_clusters=60,max_iter=5000)

labels = kmeans.fit_predict(mySeries_transformed)
time_elapsed = time.time() - since
print(f"Kmeans on PCA clustering executed in {time_elapsed // 60}m {time_elapsed % 60}s ")

297
Kmeans on PCA clustering executed in 0.0m 40.98774480819702s 


In [19]:
cluster_map = []
i = 0
cluster_x = []
cluster_dict = {}
for idx in names:
    winner_node = som.winner(full[i])
    name = f"Cluster {winner_node[0] * som_y + winner_node[1] + 1}"
    cluster_map.append((idx, name))

    cluster_x.append(name)
    i += 1

cluster_x = Counter(cluster_x)
clusters = pd.DataFrame(cluster_map, columns=["Name", "Cluster"]).sort_values(by="Cluster").set_index("Name")

In [20]:
cluster = clusters.loc["sales"].values
value = clusters["Cluster"].values
index = list(np.where(value ==cluster)[0])
names = list(clusters.index.values[index])
names.remove("sales")
names

['SMU15000007072200030SA',
 'SMU53426444245200001SA',
 'SMU45439004100000001',
 'SMU15000007072000030SA',
 'SMU53426604245200001SA',
 'SMU06125403231100001',
 'SMU21000006055000001',
 'SMU48000008081300001SA',
 'SMU48185800500000001',
 'EXP5200',
 'SMU09000006054150001SA',
 'BISM938TRADN',
 'CORP548NAN',
 'CES6056140001',
 'BISM938NRMN',
 'SMS30000009092000001',
 'SMU39104204244600001SA',
 'TURCSINFT02STSAM',
 'SMU25765249000000001SA',
 'TOTLLW2024',
 'SMU40364205553200001',
 'SMU36350044244500001SA',
 'SMU36287404100000001SA',
 'LINC731TRADN',
 'CEDA319TRADN',
 'BROW148TRADN',
 'BROW148TRAD',
 'PITGND01JPM661N',
 'CES4142000007',
 'SMU36350044244510001SA',
 'SMU24000004244510001SA',
 'CP0550LTM086NEST',
 'CP0730SIM086NEST',
 'CPIAPPNS',
 'SMU38139004200000001SA',
 'SMU41000003231100006SA',
 'SMU40000007072100001',
 'SMU38220209092000001SA',
 'SMU29411803200000030',
 'ROCH336PBSVN',
 'KALA026FIRE',
 'TX80813000M175FRBDAL',
 'LALFN',
 'CORP548NRMNN',
 'IPG3364S',
 'SMU42979626561000001S

In [93]:
corrs = {}
for name in data:
    x = data[name]
    
    corrs[abs(x.corr(target["sales"]))] = name

In [94]:
names = []
for num in sorted(corrs)[:-40:-1]:
    names.append(corrs[num])
for num in sorted(corrs)[:10:1]:
    names.append(corrs[num])

In [95]:
data = data[names]

In [96]:
full = data.join(target)

In [97]:
full

,CP0213PTM086NEST,LFEM64MAKRM647N,OWNFIRE,TOPE820PBSVN,SMU15000004322000001,SMU20928126000000001,CP0540EEM086NEST,SMU06310843133450001SA,SMU06310843133450001,SMU44772003000000030,...,HARR442URN,CGBD2564,LAUCN260190000000005,DURH537UR,NYXRPSNSA,PIEATI02CHM661N,ALAUTA1URN,ALBA513URN,JTS1000TSL,sales
0,61.03,12686900.0,2.3,8.0,2.8,63.5,69.62,38.949749,39.1,526.32,...,4.0,2.7,8168.0,4.2,17045.0,96.014029,4.4,6.1,4804.0,1308
1,61.38,12417000.0,2.3,8.1,2.8,62.6,70.65,40.001376,40.1,516.85,...,4.6,2.6,7954.0,4.4,13847.0,95.860035,4.7,6.4,4747.0,1054
2,60.48,12733700.0,2.3,8.1,2.9,65.9,70.45,40.052584,39.8,525.15,...,4.0,2.6,8518.0,4.4,14327.0,96.314645,3.3,5.9,4808.0,1258
3,59.93,12576300.0,2.3,8.4,2.9,67.9,70.67,40.071595,40.2,523.72,...,4.0,2.4,8919.0,4.3,20418.0,96.765370,3.8,5.9,4893.0,1362
4,59.94,12716000.0,2.3,8.3,2.9,68.4,70.38,39.980556,40.1,531.95,...,3.9,2.5,8417.0,4.1,19443.0,97.366404,3.4,5.8,4686.0,1225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,98.61,14101000.0,3.5,12.9,3.8,90.3,98.89,24.643202,24.9,722.71,...,4.2,2.4,8905.0,3.4,20104.0,100.422968,2.8,4.5,5352.0,5144
60,100.08,14131600.0,3.5,12.9,3.7,89.7,98.50,24.879765,24.8,763.60,...,3.6,1.8,8308.0,3.2,14871.0,100.320272,2.2,3.7,5300.0,4797
61,97.91,13939400.0,3.5,12.4,3.7,87.5,97.09,25.031696,25.0,714.56,...,4.2,2.1,8096.0,3.2,11785.0,99.940866,2.6,4.1,5366.0,5438
62,94.02,13842500.0,3.3,11.4,3.7,83.5,97.86,25.096434,24.9,720.29,...,12.1,8.3,6941.0,10.8,11493.0,99.051668,6.4,8.7,4160.0,6056


In [98]:
X, y = full.iloc[:,:-1],full.iloc[:,-1]


In [99]:
data_dmatrix = xgb.DMatrix(data=X,label=y)



In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


In [101]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3,learning_rate = 0.2,
                max_depth = 10, alpha = 5,n_estimators = 300, tree_method="gpu_hist")

In [102]:
xg_reg.fit(X_train,y_train)


[17:20:02] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=5, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=300, n_jobs=8, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=5, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='gpu_hist',
             validate_parameters=1, verbosity=None)

In [103]:
preds = xg_reg.predict(X_test)


In [104]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 858.222774


In [116]:
values = {}
for name in names:
    d = fred.get_series(name, observation_start="2/1/2021", observation_end = "2/1/2021")
    values[name] = d

In [117]:
values["sales"] = 6236
test = pd.DataFrame(values)


In [118]:
test_data = torch.from_numpy(test.to_numpy())
test_data2 = torch.from_numpy(full[:-3:-1].to_numpy())
test_data = torch.cat([test_data2, test_data]).unsqueeze(0).float()

In [119]:
model = Model()
model.load_state_dict(torch.load("model.pt"))
model(test_data)

tensor([[7341.5322]], grad_fn=<AddmmBackward>)

In [54]:
xg_reg.predict(test)

array([2555.3232], dtype=float32)

In [110]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
     

        self.output_block = nn.Sequential(nn.Conv1d(3, 16, 2, 1, 0), nn.ReLU(), 
                                          nn.Conv1d(16, 32, 2, 1, 0), nn.ReLU(), 
                                          nn.Conv1d(32, 32, 2, 1, 0), nn.ReLU(), 
                                          nn.Conv1d(32, 16, 2, 1, 0), nn.ReLU())
        self.fc = nn.Sequential(nn.LazyLinear(512), nn.ReLU(), nn.Linear(512, 256), nn.ReLU(), nn.Linear(256, 1))

    def forward(self, x):
        net = self.output_block(x)
        net = net.view(net.shape[0], -1)
        net = self.fc(net)
        
        return net


In [111]:
target[3:]

,sales
3,1362
4,1225
5,1312
6,1373
7,1530
...,...
60,4797
61,5438
62,6056
63,5773


In [112]:
class dataset(Dataset):
    def __init__(self, df, target):
        data = df[1:].reset_index(drop=True)
        sales = target[:-1].reset_index(drop=True)
        self.data = data.join(sales)
        self.target = target[3:].to_numpy()

    def __len__(self):
        return len(self.data)-3
    def __getitem__(self, idx):
        data = self.data.iloc[idx:idx+3].to_numpy()
        target = self.target[idx]
        return torch.tensor(data).float(), torch.tensor(target).float()
        
    

In [113]:
idx = int(len(data) * 0.8)
train, test = data[:idx], data[idx:]
traint, testt = target[:idx], target[idx:]
trainset = dataset(train, traint)
testset = dataset(test, testt)
train_loader = DataLoader(trainset, batch_size=8, shuffle=True)
test_loader = DataLoader(testset, batch_size=4, shuffle=True)

In [114]:
model = Model()
model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.003)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=25, factor=0.5, min_lr=0.000001)
def Train(epochs, model, train_loader):
    valid_loss_min = np.Inf
    min_rmse = np.Inf
    for epoch in range(epochs):
        train_loss = 0.0
        correct = 0.0
        total = 0.0
        valid_loss = 0.0
        rmse = 0.0
        model.train()
        for batch_idx, (images, labels) in enumerate(train_loader):
       
            images = images.cuda()
            labels = labels.cuda()
            output = model(images)
        
            loss = torch.sqrt(criterion(output,labels))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
        model.eval()
        for batch_idx, (images, labels) in enumerate(test_loader):
       
            images = images.cuda()
            labels = labels.cuda()
            output = model(images)
        
            loss = torch.sqrt(criterion(output,labels))
          
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
                
        scheduler.step(valid_loss)
        print(f"Train Loss: {train_loss.item()}")
        print(f"Valid Loss: {valid_loss.item()}")
        if valid_loss.item() < valid_loss_min:
            valid_loss_min = valid_loss.item()
            torch.save(model.state_dict(), 'model.pt')
    print(f"Min RMSE: {valid_loss_min}")

/home/dwight/miniconda3/lib/python3.8/site-packages/torch/nn/modules/lazy.py:175: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [115]:
Train(1000, model, train_loader)




Train Loss: 14404.99609375
Valid Loss: 4604.17529296875
Train Loss: 1931.1536865234375
Valid Loss: 2623.03515625
Train Loss: 1213.804443359375
Valid Loss: 4203.15966796875
Train Loss: 1327.091064453125
Valid Loss: 3934.09423828125
Train Loss: 1189.7322998046875
Valid Loss: 3725.74951171875
Train Loss: 1111.98681640625
Valid Loss: 3329.759033203125
Train Loss: 1074.5225830078125
Valid Loss: 4327.03271484375
Train Loss: 1113.3994140625
Valid Loss: 4136.115234375
Train Loss: 1087.845458984375
Valid Loss: 2974.157470703125
Train Loss: 1104.0758056640625
Valid Loss: 3163.33154296875
Train Loss: 1194.8238525390625
Valid Loss: 4032.378662109375
Train Loss: 1252.1951904296875
Valid Loss: 3607.66162109375
Train Loss: 1076.4114990234375
Valid Loss: 4027.7314453125
Train Loss: 1123.351318359375
Valid Loss: 4257.75341796875
Train Loss: 1089.855224609375
Valid Loss: 2853.810546875
Train Loss: 1039.6171875
Valid Loss: 3525.20068359375
Train Loss: 1085.1416015625
Valid Loss: 2953.9921875
Train Loss: 

Train Loss: 175.91445922851562
Valid Loss: 441.12188720703125
Train Loss: 124.5052490234375
Valid Loss: 539.479736328125
Train Loss: 151.33226013183594
Valid Loss: 546.3299560546875
Train Loss: 133.8482666015625
Valid Loss: 512.1544189453125
Train Loss: 149.49330139160156
Valid Loss: 499.51953125
Train Loss: 121.60563659667969
Valid Loss: 561.2553100585938
Train Loss: 130.08078002929688
Valid Loss: 729.8281860351562
Train Loss: 140.4468536376953
Valid Loss: 539.4808349609375
Train Loss: 176.1941375732422
Valid Loss: 493.353271484375
Train Loss: 202.536376953125
Valid Loss: 849.2951049804688
Train Loss: 167.90541076660156
Valid Loss: 530.0210571289062
Train Loss: 139.14732360839844
Valid Loss: 749.4083251953125
Train Loss: 145.4086456298828
Valid Loss: 780.8353881835938
Train Loss: 166.39984130859375
Valid Loss: 523.5712280273438
Train Loss: 150.1130828857422
Valid Loss: 446.1455078125
Train Loss: 111.23774719238281
Valid Loss: 510.6570129394531
Train Loss: 134.48744201660156
Valid Loss

Train Loss: 73.36345672607422
Valid Loss: 573.6124877929688
Train Loss: 73.79987335205078
Valid Loss: 433.02880859375
Train Loss: 72.6086654663086
Valid Loss: 563.3661499023438
Train Loss: 73.22526550292969
Valid Loss: 567.019287109375
Train Loss: 73.26133728027344
Valid Loss: 568.4072265625
Train Loss: 67.89846801757812
Valid Loss: 733.582763671875
Train Loss: 71.18912506103516
Valid Loss: 595.802734375
Train Loss: 71.85567474365234
Valid Loss: 539.2406616210938
Train Loss: 71.84952545166016
Valid Loss: 568.4303588867188
Train Loss: 68.61940002441406
Valid Loss: 571.8065795898438
Train Loss: 67.47990417480469
Valid Loss: 521.357421875
Train Loss: 69.89691162109375
Valid Loss: 564.41943359375
Train Loss: 73.46366882324219
Valid Loss: 571.046142578125
Train Loss: 71.91847229003906
Valid Loss: 544.1968383789062
Train Loss: 71.29473876953125
Valid Loss: 597.3981323242188
Train Loss: 71.07061767578125
Valid Loss: 452.64947509765625
Train Loss: 73.38777160644531
Valid Loss: 721.751037597656

Train Loss: 69.14789581298828
Valid Loss: 434.905517578125
Train Loss: 68.43154907226562
Valid Loss: 517.4584350585938
Train Loss: 69.97235107421875
Valid Loss: 569.9212036132812
Train Loss: 70.14908599853516
Valid Loss: 568.2794799804688
Train Loss: 66.25083923339844
Valid Loss: 557.3464965820312
Train Loss: 70.3773193359375
Valid Loss: 455.83154296875
Train Loss: 71.52501678466797
Valid Loss: 569.4948120117188
Train Loss: 70.58111572265625
Valid Loss: 566.4956665039062
Train Loss: 71.39920806884766
Valid Loss: 511.88525390625
Train Loss: 68.88127899169922
Valid Loss: 737.3117065429688
Train Loss: 70.20658111572266
Valid Loss: 574.5415649414062
Train Loss: 71.65664672851562
Valid Loss: 557.716552734375
Train Loss: 70.53634643554688
Valid Loss: 565.3314819335938
Train Loss: 71.50068664550781
Valid Loss: 533.237548828125
Train Loss: 69.83454132080078
Valid Loss: 544.156982421875
Train Loss: 71.62779235839844
Valid Loss: 427.2405090332031
Train Loss: 70.5545883178711
Valid Loss: 456.1021

Train Loss: 67.49808502197266
Valid Loss: 455.52789306640625
Train Loss: 67.4533920288086
Valid Loss: 568.1082763671875
Train Loss: 70.69285583496094
Valid Loss: 575.1002807617188
Train Loss: 69.1910400390625
Valid Loss: 432.73968505859375
Train Loss: 70.44413757324219
Valid Loss: 544.1473999023438
Train Loss: 69.11952209472656
Valid Loss: 519.8569946289062
Train Loss: 69.4056396484375
Valid Loss: 533.7139282226562
Train Loss: 70.09272003173828
Valid Loss: 511.24053955078125
Train Loss: 70.83734893798828
Valid Loss: 734.58935546875
Train Loss: 70.85358428955078
Valid Loss: 549.3880615234375
Train Loss: 70.39043426513672
Valid Loss: 544.1240844726562
Train Loss: 69.43566131591797
Valid Loss: 736.8877563476562
Train Loss: 69.74165344238281
Valid Loss: 724.9314575195312
Train Loss: 68.2876968383789
Valid Loss: 432.212646484375
Train Loss: 70.93346405029297
Valid Loss: 718.020263671875
Train Loss: 70.81230163574219
Valid Loss: 600.263916015625
Train Loss: 68.8253173828125
Valid Loss: 568.2

Train Loss: 68.32642364501953
Valid Loss: 599.6017456054688
Train Loss: 71.4017562866211
Valid Loss: 543.7356567382812
Train Loss: 70.52593994140625
Valid Loss: 732.484130859375
Train Loss: 70.71292877197266
Valid Loss: 567.8152465820312
Train Loss: 71.8032455444336
Valid Loss: 544.0198364257812
Train Loss: 67.22298431396484
Valid Loss: 599.4813842773438
Train Loss: 68.26220703125
Valid Loss: 544.181640625
Train Loss: 70.59828186035156
Valid Loss: 565.5779418945312
Train Loss: 69.15977478027344
Valid Loss: 600.0770263671875
Train Loss: 69.15381622314453
Valid Loss: 718.341796875
Train Loss: 70.916015625
Valid Loss: 569.7828979492188
Train Loss: 68.92594146728516
Valid Loss: 455.9560546875
Train Loss: 69.13719177246094
Valid Loss: 568.3777465820312
Train Loss: 68.3379898071289
Valid Loss: 599.6882934570312
Train Loss: 69.41172790527344
Valid Loss: 600.056640625
Train Loss: 69.88018035888672
Valid Loss: 725.142333984375
Train Loss: 70.31559753417969
Valid Loss: 737.3331298828125
Train Lo

Train Loss: 69.31764221191406
Valid Loss: 718.625
Train Loss: 70.34473419189453
Valid Loss: 534.0103149414062
Train Loss: 68.56564331054688
Valid Loss: 455.22515869140625
Train Loss: 71.22007751464844
Valid Loss: 736.667724609375
Train Loss: 69.74980926513672
Valid Loss: 569.199462890625
Train Loss: 70.56784057617188
Valid Loss: 455.437744140625
Train Loss: 68.85319519042969
Valid Loss: 426.9064025878906
Train Loss: 69.51908111572266
Valid Loss: 569.6824340820312
Train Loss: 70.21980285644531
Valid Loss: 427.0104064941406
Train Loss: 71.04505157470703
Valid Loss: 455.43939208984375
Train Loss: 66.48887634277344
Valid Loss: 728.5719604492188
Train Loss: 70.73027038574219
Valid Loss: 533.916259765625
Train Loss: 70.02530670166016
Valid Loss: 565.9945068359375
Train Loss: 70.11174774169922
Valid Loss: 600.1270141601562
Train Loss: 70.74583435058594
Valid Loss: 599.720703125
Train Loss: 70.74337768554688
Valid Loss: 548.1836547851562
Train Loss: 70.92524719238281
Valid Loss: 566.5480346679

Train Loss: 71.1082534790039
Valid Loss: 732.4857177734375
Train Loss: 69.59678649902344
Valid Loss: 599.6680297851562
Train Loss: 69.45436096191406
Valid Loss: 730.3143920898438
Train Loss: 69.72811126708984
Valid Loss: 574.667236328125
Train Loss: 69.89010620117188
Valid Loss: 543.1509399414062
Train Loss: 69.1679458618164
Valid Loss: 567.3561401367188
Min RMSE: 416.1861572265625
